# Project Part 2: Text Cleaning 

One of the 1st steps in working with twitter text is to pre-process it. The most of text data is highly unstructured and noisy - to achieve better insights or to build better algorithms, it is necessary to play with clean data. 

The possible noise elements that should be removed as follows: 


- Decoding data: transforming information from complex symbols to simple and easier to understand characters (i.e., UTF-8 encoding is widely accepted and is recommended to use)
- HTML characters: it looks like the data obtained from web usually contains a lot of html entities such as &amp in the text field, which should be removed
- URLs: URLs and hyperlinks in text data like comments, reviews, and tweets should be removed
- Emoticons: textual data can contain facial expression using characters. This information doesn't add value to build sentiment analysis model
- @mention: same with emoticons, even though it carries some information, for sentiment analysis purpose, this can be ignored
- Hash-tags: leave the text intact and just remove the '#'
- Stop-words: the commonly occurring words such as 'a', 'the', 'it', 'to' should be removed. One can either create a long list of stop-words or one can use predefined language specific libraries
- Split: check if there's any contracted or negated term i.e., 'isn't', and replace it with its original form

Examples can be found as follows. 

## Text Cleaning : Removal of Stop-words

In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
df = pd.read_csv('madrid_df.csv')
test_text = df['text'][:1]

from nltk.corpus import stopwords
import string
replace_set = stopwords.words('english')
no_stopwords = test_text.str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in replace_set))

print test_text
print no_stopwords

0    Countdown to @ChampionsLeague 2018 Final in Kyiv, Russia. @LFC vs @realmadrid #YNWA #6DaysToGo https://t.co/Ixx0jnPdNy
Name: text, dtype: object
0    Countdown @ChampionsLeague 2018 Final Kyiv, Russia. @LFC vs @realmadrid #YNWA #6DaysToGo https://t.co/Ixx0jnPdNy
Name: text, dtype: object


## Text Cleaning : Decoding and Escaping HTML 

In [5]:
test_text2 = df['text'][83]
print(test_text2)
test_text2_decoding = test_text2.decode("utf8").encode('ascii','ignore')

import HTMLParser
html_parser = HTMLParser.HTMLParser()
no_html = html_parser.unescape(test_text2_decoding)
print(no_html)

Have a read of the latest Q&amp;A from @_Chris_Hurst_ on @LiverpoolFCHQ_ 🔴🇺🇦 https://t.co/VlAFdgPy4s
Have a read of the latest Q&A from @_Chris_Hurst_ on @LiverpoolFCHQ_  https://t.co/VlAFdgPy4s


## Text Cleaning : URL links

In [6]:
import re

no_urls = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+','',no_html)
print(no_urls)

Have a read of the latest Q&A from @_Chris_Hurst_ on @LiverpoolFCHQ_  


## Text Cleaning : @mention

In [ ]:
no_atmention = re.sub(r'@[\w_]+','', no_urls)
print(no_atmention)

## Text Cleaning : Emoticons

In [7]:
test_text3 = df['text'][11]
print(test_text3)

re.sub(r'[:=;] [oO\-]?[D\)\]\(\]/\\OpP]', '', test_text3)

@socraticjuan How could @realmadrid do that atrocity to @IkerCasillas i feel sorry for him 😠😠😢😢


'@socraticjuan How could @realmadrid do that atrocity to @IkerCasillas i feel sorry for him \xf0\x9f\x98\xa0\xf0\x9f\x98\xa0\xf0\x9f\x98\xa2\xf0\x9f\x98\xa2'

## Defining Text Cleaning Function

Now definding a function for text cleaning and apply to the whole dataset. Tokenization, stemming/lemmatization will be dealt with later stage when creating matrix with either count vectorizer or Tfidf vectorizer.

In [8]:
df_madrid = pd.read_csv('madrid_df.csv')
demo = df_madrid['text'][:5] # demo testing with 10 texts 
demo

0    Countdown to @ChampionsLeague 2018 Final in Kyiv, Russia. @LFC vs @realmadrid #YNWA #6DaysToGo https://t.co/Ixx0jnPdNy                      
1    ¡@Cristiano llega a los 4⃣5⃣0⃣ goles con el @realmadrid! 💥 https://t.co/YHfrJ3E6B3                                                         
2    ¡@Cristiano llega a los 4⃣5⃣0⃣ goles con el @realmadrid ! 💥 https://t.co/NTLogiPwl1                                                        
3    Well, @realmadrid ended this seasons @LaLiga campaign the same way they started it, shitty. I just hope that Zidane… https://t.co/zGxANmIWu7
4    @BleacherReport @Cristiano @realmadrid Yall late https://t.co/psBUG5iAdB                                                                    
Name: text, dtype: object

In [9]:
replace_set = stopwords.words('english') 
demo = demo.str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in replace_set)) # texts with no stop-words
demo

C:\Users\yaffy\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


0    Countdown @ChampionsLeague 2018 Final Kyiv, Russia. @LFC vs @realmadrid #YNWA #6DaysToGo https://t.co/Ixx0jnPdNy
1    ¡@Cristiano llega los 4⃣5⃣0⃣ goles con el @realmadrid! 💥 https://t.co/YHfrJ3E6B3                               
2    ¡@Cristiano llega los 4⃣5⃣0⃣ goles con el @realmadrid ! 💥 https://t.co/NTLogiPwl1                              
3    Well, @realmadrid ended seasons @LaLiga campaign way started it, shitty. I hope Zidane… https://t.co/zGxANmIWu7 
4    @BleacherReport @Cristiano @realmadrid Yall late https://t.co/psBUG5iAdB                                        
Name: text, dtype: object

Splitting terms with contraction and negation 

In [11]:
split_dic = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
             "can't've": "cannot have", "'cause": "because", "could've": "could have", 
             "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
             "doesn't": "does not", "don't": "do not", "hadn't": "had not",  
             "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
             "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
             "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
             "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
             "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
             "I'll've": "I will have","I'm": "I am", "I've": "I have", 
             "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
             "i'll've": "i will have","i'm": "i am", "i've": "i have", 
             "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
             "it'll": "it will", "it'll've": "it will have","it's": "it is", 
             "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
             "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
             "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
             "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
             "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
             "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
             "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
             "she's": "she is", "should've": "should have", "shouldn't": "should not", 
             "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
             "this's": "this is",
             "that'd": "that would", "that'd've": "that would have","that's": "that is", 
             "there'd": "there would", "there'd've": "there would have","there's": "there is", 
             "here's": "here is",
             "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
             "they'll've": "they will have", "they're": "they are", "they've": "they have",
             "to've": "to have", "wasn't": "was not", "we'd": "we would", 
             "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
             "we're": "we are", "we've": "we have", "weren't": "were not",
             "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
             "what's": "what is", "what've": "what have", "when's": "when is", 
             "when've": "when have", "where'd": "where did", "where's": "where is", 
             "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
             "who's": "who is", "who've": "who have", "why's": "why is", 
             "why've": "why have", "will've": "will have", "won't": "will not",
             "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
             "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
             "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
             "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
             "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [12]:
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
import re
from bs4 import BeautifulSoup

tok = WordPunctTokenizer()

pat1 = r'@[\w_]+' # @-mention
pat2 = r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+' # URLs
pat3 = r'[:=;] [oO\-]?[D\)\]\(\]/\\OpP]' # emoticons
pat4 = r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)' # hash-tags
pat5 = r'www.[^ ]+' # additions to URLs, texts with 'www..'
combined_pat = r'|'.join((pat1, pat2, pat3,pat4,pat5))


split_pattern = re.compile(r'\b(' + '|'.join(split_dic.keys()) + r')\b')


def tweet_cleaner(demo):
    soup = BeautifulSoup(demo, 'lxml') # HTML
    souped = soup.get_text()
    try:
        clean = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
        # decoding text with 'utf-8-sig'
    except:
        clean = souped
    
    stripped = re.sub(combined_pat, '', clean)
    lower_case = stripped.lower()
    split_handled = split_pattern.sub(lambda x: split_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", split_handled)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', letters_only)
    # check if there's any spell with repeated characters such as 'soooo good', and transform it as 'soo good' or 'loove it'. Not a perfect solution but could reduct feature space by making
    words = [x for x in tok.tokenize(spell_corrected) if len(x) > 1]

    return (" ".join(words)).strip()

test_result = []
for t in demo:
    test_result.append(tweet_cleaner(t))
    # [tweet_cleaner(t) for t in demo]

test_result # cleaning texts

C:\Users\yaffy\Anaconda2\lib\encodings\utf_8_sig.py:19: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if input[:3] == codecs.BOM_UTF8:


[u'countdown final kyiv russia vs',
 u'llega los goles con el',
 u'llega los goles con el',
 u'well ended seasons campaign way started it shitty hope zidane',
 u'yall late']

## Apply Text Cleaning Function to Tweets

In [13]:
df_madrid = pd.read_csv('madrid_df.csv')
replace_set = stopwords.words('english') 
df_madrid['text'] = df_madrid['text'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in replace_set)) # replace stop-words

C:\Users\yaffy\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
df_madrid['clean_text'] = [tweet_cleaner(t) for t in df_madrid.text]

C:\Users\yaffy\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/oEJ7354EcO" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\yaffy\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/NZGnT2XvKu" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\yaffy\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/GOs8YOaLFe" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to

After cleaning the tweets with cleaner function, I create a new dataframe with clean text and certain columns that will be used in the further analysis

In [15]:
col_to_drop = ['text', 'extended_tweet','ideo_score', 'friends', 'followers', 'listed', 'screen_name', 'id', 'name']
clean_madrid = df_madrid.drop(col_to_drop, axis=1)

3,777 entries have null entries for the clean_text column. Looking at the original dataframe, those texts either with Twitter ID or URL address are identified as NaN. These are the info I decide to discard for the sentiment analysis, so I will drop these null rows, and update the data frame.

In [16]:
clean_madrid['clean_text'].replace('', np.nan, inplace=True)
print(clean_madrid['clean_text'].isna().sum())

df_madrid.iloc[clean_madrid[clean_madrid.isna().any(axis=1)].index,:].head()

3777


,created_at,name,screen_name,id,friends,followers,listed,text,extended_tweet,ideo_score,clean_text
10,Mon May 21 05:26:15 +0000 2018,Gerson Arce,19gerson,463365631.0,675,786,3,E L C O M A N D A N T E\r\r\r\n4⃣5⃣0⃣⚽ https://t.co/0NHlSdQLp9,NaN,NaN,
13,Mon May 21 06:21:00 +0000 2018,Rana Ghosal,ranaaissance,56296160.0,532,456,46,@kingpatprash @ChampionsLeague @realmadrid @LFC @sportmarketing1 @Sports_Managing @kennethcortsen @AJKarg… https://t.co/gFIrSNuPJJ,"{u'display_text_range': [133, 162], u'entities': {u'user_mentions': [{u'indices': [0, 13], u'screen_name': u'kingpatprash', u'id': 350920112, u'name': u'Prashant Pathare', u'id_str': u'350920112'}, {u'indices': [14, 30], u'screen_name': u'ChampionsLeague', u'id': 627673190, u'name': u'UEFA Champions League', u'id_str': u'627673190'}, {u'indices': [31, 42], u'screen_name': u'realmadrid', u'id': 14872237, u'name': u'Real Madrid C.F. \u26bd\ufe0f', u'id_str': u'14872237'}, {u'indices': [43, 47], u'screen_name': u'LFC', u'id': 19583545, u'name': u'Liverpool FC', u'id_str': u'19583545'}, {u'indices': [48, 64], u'screen_name': u'sportmarketing1', u'id': 27426996, u'name': u'alan seymour', u'id_str': u'27426996'}, {u'indices': [65, 81], u'screen_name': u'Sports_Managing', u'id': 873554509037015040L, u'name': u'\xd3scar Y\xe1\xf1ez', u'id_str': u'873554509037015040'}, {u'indices': [82, 97], u'screen_name': u'kennethcortsen', u'id': 24509215, u'name': u'Kenneth Cortsen', u'id_str': u'24509215'}, {u'indices': [98, 105], u'screen_name': u'AJKarg', u'id': 28756684, u'name': u'Adam Karg', u'id_str': u'28756684'}, {u'indices': [106, 121], u'screen_name': u'geoffwnjwilson', u'id': 109710458, u'name': u'Geoff Wilson', u'id_str': u'109710458'}, {u'indices': [122, 132], u'screen_name': u'Lu_Class_', u'id': 2955644493L, u'name': u'\u0141ukasz B\u0105czek', u'id_str': u'2955644493'}], u'symbols': [], u'hashtags': [], u'urls': []}, u'full_text': u'@kingpatprash @ChampionsLeague @realmadrid @LFC @sportmarketing1 @Sports_Managing @kennethcortsen @AJKarg @geoffwnjwilson @Lu_Class_ Very interesting. Optimistic?'}",NaN,
16,Mon May 21 07:06:58 +0000 2018,Sebastian Yuen ツ,S3bster,37780157.0,3136,3398,93,@boltyboy @Health2eu @bluetopaz @himsseurope @ChampionsLeague @lucienengelen @pascal_lardier @anoizet @janedwelly… https://t.co/5AWVfVgYyy,"{u'display_text_range': [196, 339], u'entities': {u'user_mentions': [{u'indices': [0, 9], u'screen_name': u'boltyboy', u'id': 1248081, u'name': u'Matthew Holt', u'id_str': u'1248081'}, {u'indices': [10, 20], u'screen_name': u'Health2eu', u'id': 221140787, u'name': u'Health 2.0 Europe', u'id_str': u'221140787'}, {u'indices': [21, 31], u'screen_name': u'bluetopaz', u'id': 1073381, u'name': u'Indu Subaiya', u'id_str': u'1073381'}, {u'indices': [32, 44], u'screen_name': u'himsseurope', u'id': 132204567, u'name': u'HIMSS Europe', u'id_str': u'132204567'}, {u'indices': [45, 61], u'screen_name': u'ChampionsLeague', u'id': 627673190, u'name': u'UEFA Champions League', u'id_str': u'627673190'}, {u'indices': [62, 76], u'screen_name': u'lucienengelen', u'id': 15123270, u'name': u'Lucien Engelen', u'id_str': u'15123270'}, {u'indices': [77, 92], u'screen_name': u'pascal_lardier', u'id': 78607265, u'name': u'Pascal Lardier', u'id_str': u'78607265'}, {u'indices': [93, 101], u'screen_name': u'anoizet', u'id': 312960789, u'name': u'Aline Noizet', u'id_str': u'312960789'}, {u'indices': [102, 113], u'screen_name': u'janedwelly', u'id': 108297471, u'name': u'Jane Dwelly', u'id_str': u'108297471'}, {u'indices': [114, 130], u'screen_name': u'Barry_HealthTec', u'id': 44844855, u'name': u'Barry Shrier', u'id_str': u'44844855'}, {u'indices': [131, 144], u'screen_name': u'HealthEugene', u'id': 16139549, u'name': u'Eugene Borukhovich', u'id_str': u'16139549'}, {u'indices': [145, 158], u'screen_name': u'chrispointon', u'id': 28578944, u'name': u'Chris Pointon', u'id_str': u'28578944'}, {u'indices': [159, 170], u'screen_name': u'Anniecoops', u'id': 20327119, u'name': u'Anne Cooper RN FQNI', u'i

In [17]:
clean_madrid.dropna(inplace=True)
clean_madrid.reset_index(drop=True,inplace=True)

In [18]:
print(clean_madrid.info())
clean_madrid.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31233 entries, 0 to 31232
Data columns (total 2 columns):
created_at    31233 non-null object
clean_text    31233 non-null object
dtypes: object(2)
memory usage: 488.1+ KB
None


,created_at,clean_text
0,Mon May 21 01:44:58 +0000 2018,countdown final kyiv russia vs
1,Mon May 21 04:01:00 +0000 2018,llega los goles con el
2,Mon May 21 04:08:40 +0000 2018,llega los goles con el
3,Mon May 21 04:38:32 +0000 2018,well ended seasons campaign way started it shitty hope zidane
4,Mon May 21 04:52:52 +0000 2018,yall late


As mentioned ealier, the data was collected over four weeks since from 21 May and end at 16 June 2018 at pre-, mid- and post-event. Now I will split the texts according to the date. It can be found at the new column 'period': 

- 1: pre-event, 2018-05-21 - 2018-05-25
- 2: mid-event, 2018-05-26
- 3: post-event (week one), 2018-05-27 - 2018-06-03
- 4: post-event (week two), 2018-06-04 - 2018-06-10
- 5: post-event (week three), 2018-06-11 - 2018-06-16

Change the datetime format and keep the date part only (e.g., change 'Mon May 21 01:44:58 +0000 2018' to '2018-05-21')

In [19]:
clean_madrid['created_at'] = pd.to_datetime(clean_madrid.created_at).dt.date
clean_madrid['created_at'] = pd.to_datetime(clean_madrid.created_at, errors='coerce')

Generate a new column named 'period' and set the column values based on dates as descripted above. Save the cleaning texts with five-periods to a new dataset 'clean_madrid.csv'

In [21]:
clean_madrid['period'] = ' '
 
index1 = clean_madrid.loc[(clean_madrid['created_at'] >= '2018-05-21') & (clean_madrid['created_at'] <= '2018-05-25')].index
index2 = clean_madrid.loc[clean_madrid['created_at'] == '2018-05-26'].index
index3 = clean_madrid.loc[(clean_madrid['created_at'] >= '2018-05-27') & (clean_madrid['created_at'] <= '2018-06-03')].index
index4 = clean_madrid.loc[(clean_madrid['created_at'] >= '2018-06-04') & (clean_madrid['created_at'] <= '2018-06-10')].index
index5 = clean_madrid.loc[(clean_madrid['created_at'] >= '2018-06-11') & (clean_madrid['created_at'] <= '2018-06-16')].index
 
clean_madrid.loc[index1, 'period'] = 'pre-event'
clean_madrid.loc[index2, 'period'] = 'mid-event'
clean_madrid.loc[index3, 'period'] = 'post-event-week1'
clean_madrid.loc[index4, 'period'] = 'post-event-week2'
clean_madrid.loc[index5, 'period'] = 'post-event-week3'

clean_madrid['team'] = 'RMadrid'
clean_madrid = clean_madrid.drop(columns = ['created_at'])

clean_madrid.head()
clean_madrid.to_csv('clean_madrid.csv')

,clean_text,period,team
0,countdown final kyiv russia vs,pre-event,RMadrid
1,llega los goles con el,pre-event,RMadrid
2,llega los goles con el,pre-event,RMadrid
3,well ended seasons campaign way started it shitty hope zidane,pre-event,RMadrid
4,yall late,pre-event,RMadrid


Apply text cleaning function to Liverpool dateset and save as clean_liverp.csv

In [24]:
print(clean_liverp.info())
clean_liverp.head()

clean_liverp = to_csv('clean_liverp.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145504 entries, 0 to 145503
Data columns (total 3 columns):
clean_text    135076 non-null object
period        145504 non-null object
team          145504 non-null object
dtypes: object(3)
memory usage: 3.3+ MB
None


,clean_text,period,team
0,sad news justify anybody religious may ask,pre-event,Liverpool
1,why would want de vrij or yaya toure,pre-event,Liverpool
2,follow,pre-event,Liverpool
3,he already signed for inter,pre-event,Liverpool
4,de vrij match fixed inters win could play cl doubt wants lfc,pre-event,Liverpool
